## Modelo matemático exato

<br>

<b>Índices:</b>

$i = 1,...,m$

$j = 1,...,n$

<br>

<b>Parâmetros:</b>

$d_{ij} = \text{distância entre os pontos } i \text{ e } j$

$c_i = \text{número de estudantes no ponto }i$

$p = \text{número de medianas}$

<br>

<b>Variáveis de decisão:</b>

$
    y_{j}=
    \begin{cases}
      1, & \text{se o ponto } j \text{ é escolhido como mediana} \\
      0, & \text{caso contrário}
    \end{cases}
$

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o ponto } i \text{ é alocado à mediana } j \\
      0, & \text{caso contrário}
    \end{cases}
$

####  Modelo de Programação Linear Inteira:

$\text{min }z = \sum\limits_{i=1}^{m} \sum\limits_{j=1}^{n} c_i d_{ij} x_{ij}$

sujeito a:

$\sum\limits_{j=1}^{n} y_{j} = p$

$\sum\limits_{j=1}^{n} x_{ij} = 1, \forall \; i$

$x_{ij} \leq y_j, \forall \; i,j$

$y_j \in \{0,1\}, \forall \; j$

$x_{ij} \in \{0,1\}, \forall \; i,j$

In [1]:
# Carrega as bibliotecas necessárias:

import pyomo.environ as pyo
import pandas as pd

In [2]:
# Declara o modelo computacional:

modelo = pyo.AbstractModel()

# Parâmetros auxiliares:
modelo.distritos = pyo.Param()
modelo.terrenos = pyo.Param()
modelo.estudantes = pyo.Param(within=pyo.Any)
modelo.distancias = pyo.Param(within=pyo.Any)

# Índices:
modelo.I = pyo.RangeSet(modelo.distritos)
modelo.J = pyo.RangeSet(modelo.terrenos)

# Parâmetros:
modelo.d = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: modelo.distancias()[i-1][j-1])
modelo.c = pyo.Param(modelo.I, initialize=lambda modelo, i: modelo.estudantes()[i-1])
modelo.p = pyo.Param()

# Variáveis de decisão:
modelo.y = pyo.Var(modelo.J, within=pyo.Binary)
modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.Binary)

# Função objetivo:
def f_obj(modelo):
    return sum(modelo.x[i,j] * modelo.d[i,j] * modelo.c[i] for i in modelo.I for j in modelo.J)
modelo.z = pyo.Objective(rule=f_obj, sense=pyo.minimize)

# Restrições:

def f_restr1(modelo, j):
    return sum(modelo.y[j] for j in modelo.J) == modelo.p()
modelo.restr_1 = pyo.Constraint(modelo.J, rule=f_restr1)

def f_restr2(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.J) == 1
modelo.restr_2 = pyo.Constraint(modelo.I, rule=f_restr2)

def f_restr3(modelo, i, j):
    return modelo.x[i,j] <= modelo.y[j]
modelo.restr_3 = pyo.Constraint(modelo.I, modelo.J, rule=f_restr3)

In [3]:
# Lê os dados de entrada:

dados_entrada = pyo.DataPortal()
dados_entrada.load(filename='instancia1.json')

In [4]:
# Cria a instância:

instancia = modelo.create_instance(dados_entrada)

In [5]:
# Resolve a instância:

resultado = pyo.SolverFactory('glpk').solve(instancia)
print(resultado)


Problem: 
- Name: unknown
  Lower bound: 6830.0
  Upper bound: 6830.0
  Number of objectives: 1
  Number of constraints: 35
  Number of variables: 29
  Number of nonzeros: 89
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.038720130920410156
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [6]:
instancia.y.pprint()

y : Size=4, Index=J
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary


In [7]:
instancia.x.pprint()

x : Size=24, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :   1.0 :     1 : False : False : Binary
    (1, 2) :     0 :   0.0 :     1 : False : False : Binary
    (1, 3) :     0 :   0.0 :     1 : False : False : Binary
    (1, 4) :     0 :   0.0 :     1 : False : False : Binary
    (2, 1) :     0 :   0.0 :     1 : False : False : Binary
    (2, 2) :     0 :   1.0 :     1 : False : False : Binary
    (2, 3) :     0 :   0.0 :     1 : False : False : Binary
    (2, 4) :     0 :   0.0 :     1 : False : False : Binary
    (3, 1) :     0 :   1.0 :     1 : False : False : Binary
    (3, 2) :     0 :   0.0 :     1 : False : False : Binary
    (3, 3) :     0 :   0.0 :     1 : False : False : Binary
    (3, 4) :     0 :   0.0 :     1 : False : False : Binary
    (4, 1) :     0 :   0.0 :     1 : False : False : Binary
    (4, 2) :     0 :   1.0 :     1 : False : False : Binary
    (4, 3) :     0 :   0.0 :     1 : False : False : Binary
    (4, 4) : 